In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds.npy')
means = np.load('../Data/means.npy')
long_normal_value = np.load('../Data/long_normal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [6]:
# baseline model
# yhat = long_value[:,803-61:803-60]
# y = long_value[:,803-60:]
# np.mean(np.abs(y - yhat))
# >> 373.9036
# SMAPE_np(y,yhat)
# >> 42.56
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 77.51

In [5]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [6]:
conv = Sequential(ConvBatchRelu1D(3,8,3),
                  ConvBatchRelu1D(8,16,3),
                  ConvBatchRelu1D(16,32,3),
                  ConvBatchRelu1D(32,64,3))

In [7]:
rnn = GRU(64,64,2,batch_first=True)

In [8]:
linear = Sequential(Linear(64,64),LeakyReLU(0.1,True),Linear(64,1),LeakyReLU(0.1,True))

In [9]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized(conv,rnn,linear).to('cuda:0')

In [10]:
#loss_func = loss_func_generator(MSELoss())
#loss_func = loss_func_generator(L1Loss())
loss_func_nor = loss_func_generator_normalized(SMAPE)

In [12]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [13]:
model_nor = fit(5, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:92.13075620028638, val_loss:72.54814147949219
epoch:1, train_loss:87.91607714638191, val_loss:71.5727310180664
epoch:2, train_loss:87.34886222404519, val_loss:71.88652801513672
epoch:3, train_loss:87.17471145471761, val_loss:71.52730560302734
epoch:4, train_loss:87.22566221365656, val_loss:71.30207824707031
Training completed in 131.87282967567444s


In [14]:
model_nor = fit(2, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip)

epoch:0, train_loss:87.29059219755656, val_loss:71.83431243896484
epoch:1, train_loss:87.06253706383582, val_loss:70.98779296875
Training completed in 52.05903673171997s


In [16]:
model = CNN_RNN2seq(conv,rnn,linear).to('cuda:0')

In [17]:
opt = Adam(trainable_parameter(model),lr=3e-4)

In [18]:
loss_func = loss_func_generator(SMAPE)

In [19]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:41.59671775442331, val_loss:44.9703254699707
epoch:1, train_loss:41.84212299801525, val_loss:44.17484664916992
epoch:2, train_loss:41.981038964227075, val_loss:44.97205352783203
epoch:3, train_loss:42.11262688513247, val_loss:44.011470794677734
epoch:4, train_loss:42.14127385925135, val_loss:44.027320861816406
Training completed in 143.91382765769958s
